In [1]:
# The Goal is to take a pdb file and identify all the WC base pairs and the hydrogen bonded atoms(?)
# Then create an input file in Plumed's format to define the distances between those hydrogen bonded atoms so
# that a constraints/restraint function can be added later

# This is specifically for CD44 SS_B right now though can be reworked for other uses

# Running Plumed with these intial constraints/restraints will then allow me to reduce the phase space and 
# then be able to reduce computational run time

# Inputs: .pdb file
# Return: some plumed input file (?)

# Made by Isabel Dengos (ijdengos@gmail.com)
# October 2021
# Chen Lab, RNA Institute, UAlbany

In [2]:
# Import all packages needed
from simtk.openmm.app import PDBFile
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import pandas as pd
import numpy as np
import sys

In [3]:
# Import all files needed
pdb = PDBFile("/network/rit/home/ID653677/ChenRNALab/idengos/CD44/SS_B/SS_B_nores/SS_B.pdb")

In [4]:
#beginig of Plumed output build
k = 0

lines = ['#SETTINGS MOLFILE=/network/rit/home/ID653677/ChenRNALab/idengos/CD44/SS_B/SS_B_nores/SS_B.pdb', 
         '# vim:ft=plumed', 'MOLINFO STRUCTURE=SS_B.pdb']

with open('plumed.dat', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
        f.close
        
#for checking in pymol to see if okay later
for_pymol = 'select'

In [5]:
# Use OpenMM to take out the residues?
top = pdb.getTopology()
pos = pdb.getPositions()
# Match up which residue with which
#Rememeber resideues are n-1



In [6]:
#Making lists of residue pairs for the structure /(believe me this was the easist way since 40 was missing)
left_side = [*range(0,12), *range(17,26)]
right_side = [*range(47,40,-1), *range(39,25,-1)]

#Check to make sure your lists are what you need for your particular structure
print(left_side)
print(right_side)

#check that they're the same length
print(len(left_side))
print(len(right_side))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[47, 46, 45, 44, 43, 42, 41, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26]
21
21


In [7]:

for t in range(len(left_side)):
    L = left_side[t]
    R = right_side[t]
    
#for debugging GU pair
#for t in range(1,2):
 #   L=2
  #  R=45
    resLeft = list(top.residues())[L]
    atom_idsLeft = []
    for atom in resLeft.atoms():
        atom_idsLeft.append(atom.id)
    #print(atom_idsLeft)

    resRight = list(top.residues())[R]
    atom_idsRight = []
    for atom in resRight.atoms():
        atom_idsRight.append(atom.id)
    #print(atom_idsRight)


# if first list hydrogen, then look at second list and see if "close" to another, if not hydorgen, call pair

    first_H = []
    n_list = []
    for atom in resLeft.atoms():
        if atom.element.symbol == 'H':
            n_list.append(atom.id)
            first_H.append(pos[atom.index])
            #first_H = pos[atom.index]

            #go through second list and check distance
            #should probably do the distance for all of them just to check

    non_H = []
    m_list = []
    for atom in resRight.atoms():
        if atom.element.symbol != 'H':
            m_list.append(atom.id)
            non_H.append(pos[atom.index])



    for i in range(len(first_H)):

        for j in range(len(non_H)): 

            dist = np.linalg.norm(first_H[i]-non_H[j])
            if dist < .25*nanometers:

                newl = 'd{:d}: DISTANCE ATOMS={:d},{:d}'.format(k,int(n_list[i]), int(m_list[j]))
                p1newl = ' id {:d} or id {:d} or'.format(int(n_list[i]), int(m_list[j])) #for pymol easy access
                for_pymol += p1newl
                
                with open('plumed.dat', 'a') as f:
                    f.write('\n')
                    f.write(newl)
                k = k+1

    #Second list Hydrogen

    sfirst_H = []
    sn_list = []
    for atom in resRight.atoms():
        if atom.element.symbol == 'H':
            sn_list.append(atom.id)
            sfirst_H.append(pos[atom.index])
            #first_H = pos[atom.index]

            #go through second list and check distance
            #should probably do the distance for all of them just to check

    snon_H = []
    sm_list = []
    for atom in resLeft.atoms():
        if atom.element.symbol != 'H':
            sm_list.append(atom.id)
            snon_H.append(pos[atom.index])




    for i in range(len(sfirst_H)):

        for j in range(len(snon_H)): 

            sdist = np.linalg.norm(sfirst_H[i]-snon_H[j])
            if sdist < .25*nanometers:

                snewl = 'd{:d}: DISTANCE ATOMS={:d},{:d}'.format(k,int(sn_list[i]), int(sm_list[j]))
                pnewl = ' id {:d} or id {:d} or'.format(int(sn_list[i]), int(sm_list[j])) #for pymol easy access
                for_pymol += pnewl
                
                with open('plumed.dat', 'a') as f:
                    f.write('\n')
                    f.write(snewl)
                k = k+1           

                
     #print(first_H[i])

    #diatnce_vect = np.linalg.norm(first_H[i]/nanometer-non_H/nanometers)
    #print(diatnce_vect)



            #pick out the one; break if there is more than one

            #append a "n,m" list that can be used to read onto a plumed file
       # print(atom.name)

    #for atom in res47.atoms():
     #   print(atom.name)    

    #same for second list


#Then which atom goes with which (thining some type of: "not on your own residue"
#needs to be a heavy atom-hydrogen and then within certain distance and then boom ya'll are besties)
#probably make those pairs into an array or little list

In [8]:
print(for_pymol)



select id 82 or id 1447 or id 1449 or id 84 or id 116 or id 1417 or id 1419 or id 118 or id 151 or id 1389 or id 1385 or id 146 or id 1387 or id 145 or id 1387 or id 146 or id 1387 or id 147 or id 182 or id 1355 or id 1351 or id 177 or id 1353 or id 176 or id 1353 or id 177 or id 1353 or id 178 or id 213 or id 1323 or id 1321 or id 209 or id 246 or id 1260 or id 1256 or id 241 or id 1258 or id 240 or id 1258 or id 241 or id 1258 or id 242 or id 274 or id 1220 or id 274 or id 1222 or id 1226 or id 276 or id 307 or id 1195 or id 1193 or id 303 or id 337 or id 1165 or id 1163 or id 335 or id 367 or id 1130 or id 369 or id 1131 or id 1135 or id 371 or id 589 or id 1072 or id 1070 or id 587 or id 617 or id 1036 or id 622 or id 1040 or id 1038 or id 618 or id 655 or id 1008 or id 1004 or id 650 or id 1006 or id 649 or id 1006 or id 650 or id 1006 or id 651 or id 683 or id 970 or id 974 or id 685 or id 713 or id 938 or id 715 or id 937 or id 715 or id 938 or id 715 or id 939 or id 943 or id 7

In [9]:
# Then we write into a text file using a loop into plumed's format
#So like start the text file and then use a loop to write the pairs and the d(i): n,m blahhh
# need to use atom.id for each not # beacuse that will give probably actually like what I need hopefully lol